In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
import os

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量

# 讀取資料並檢視
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# 對 label 進行 one-hot encoding (y_trian 原本是純數字)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


## 首先我們使用一般的 DNN (MLP) 來訓練
由於 DNN 只能輸入一維的資料，我們要先將影像進行攤平，若 (50000, 32, 32, 3) 的影像，攤平後會變成 (50000, 32*32*3) = (50000, 3072)

In [4]:
# 將資料攤平成一維資料
x_train = x_train.reshape(50000, 3072) 
x_test = x_test.reshape(10000, 3072)

# 將資料變為 float32 並標準化
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


In [5]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(3072,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 1,841,162
Trainable params: 1,841,162
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [7]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 2s 47us/step - loss: 2.1391 - accuracy: 0.2513 - val_loss: 1.8273 - val_accuracy: 0.3255
Epoch 2/10
50000/50000 [==============================] - 2s 38us/step - loss: 1.8582 - accuracy: 0.3290 - val_loss: 1.8425 - val_accuracy: 0.3354
Epoch 3/10
50000/50000 [==============================] - 2s 39us/step - loss: 1.7807 - accuracy: 0.3617 - val_loss: 1.7241 - val_accuracy: 0.3889
Epoch 4/10
50000/50000 [==============================] - 2s 41us/step - loss: 1.7313 - accuracy: 0.3788 - val_loss: 1.6599 - val_accuracy: 0.4167
Epoch 5/10
50000/50000 [==============================] - 2s 39us/step - loss: 1.6965 - accuracy: 0.3925 - val_loss: 1.6466 - val_accuracy: 0.4065
Epoch 6/10
50000/50000 [==============================] - 2s 37us/step - loss: 1.6668 - accuracy: 0.4039 - val_loss: 1.6354 - val_accuracy: 0.4189
Epoch 7/10
50000/50000 [==============================] - 2s 38us/st

In [8]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.5553699243545531
Test accuracy: 0.45190000534057617


## 接下來我們使用 CNN 來訓練神經網路
CNN 的原理非常適合處理影像類的資料，就讓我們來看看，同樣的訓練條件，CNN 是否顯著優於 DNN 呢?

In [9]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [10]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)       

In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [12]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 9s 185us/step - loss: 1.7459 - accuracy: 0.3664 - val_loss: 1.3795 - val_accuracy: 0.4977
Epoch 2/10
50000/50000 [==============================] - 7s 132us/step - loss: 1.3090 - accuracy: 0.5345 - val_loss: 1.1039 - val_accuracy: 0.6082
Epoch 3/10
50000/50000 [==============================] - 7s 132us/step - loss: 1.0992 - accuracy: 0.6122 - val_loss: 1.0550 - val_accuracy: 0.6312
Epoch 4/10
50000/50000 [==============================] - 7s 135us/step - loss: 0.9651 - accuracy: 0.6625 - val_loss: 0.8497 - val_accuracy: 0.7062
Epoch 5/10
50000/50000 [==============================] - 7s 134us/step - loss: 0.8738 - accuracy: 0.6943 - val_loss: 0.7706 - val_accuracy: 0.7333
Epoch 6/10
50000/50000 [==============================] - 7s 134us/step - loss: 0.8055 - accuracy: 0.7208 - val_loss: 0.7799 - val_accuracy: 0.7305
Epoch 7/10
50000/50000 [==============================] - 7s 1

In [13]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.7308754851341247
Test accuracy: 0.7498000264167786


## 同樣運算 10 個 epochs，但 CNN 在 test data 的準確率顯著優於 DNN!

## 作業

### 1. 請試著調整各個超參數，並說明那些超參數對於結果有明顯的影響?
> ANS: CNN可調整的參數有：Conv層數, filter大小及數量, dropout比率, padding的數值.
> 其中Conv的層數影響比較大

### 2. CNN 與 DNN 哪個模型的參數數量比較多? 造成參數的數量不同的原因在哪?

> ANS: DNN 參數數量比較多

| CNN | DNN |
|--- | --- |
| 1,250,858 | 1,841,162| 

> ANS: CNN 有Convolutional層，它是一個kernel/filter。主要可以將各別數值分斂成相對有幾何意義的特徵。